In [1]:
import json

from rlcache.backend import storage_from_config
from rlcache.cache_constants import CacheInformation, OperationType
from rlcache.observer import ObservationType
from rlcache.strategies.strategies_from_config import strategies_from_config
%load_ext autoreload
%autoreload 2

""" TODO later
DATABASE_BACKEND = storage_from_config(CONFIG['database_backend_settings'])

CACHE_BACKEND = storage_from_config(CONFIG['cache_backend_settings'])
CACHE_MANAGER = CacheManager(config=CONFIG['cache_manager_settings'],
                             cache=CACHE_BACKEND,
                             backend=DATABASE_BACKEND)
REQUESTS_COUNTER = Counter()
"""

" TODO later\nDATABASE_BACKEND = storage_from_config(CONFIG['database_backend_settings'])\n\nCACHE_BACKEND = storage_from_config(CONFIG['cache_backend_settings'])\nCACHE_MANAGER = CacheManager(config=CONFIG['cache_manager_settings'],\n                             cache=CACHE_BACKEND,\n                             backend=DATABASE_BACKEND)\nREQUESTS_COUNTER = Counter()\n"

In [28]:
config_file = 'configs/rl_caching_strategy.json'
with open(config_file, 'r') as fp:
    CONFIG = json.load(fp)

exp_name = CONFIG['experiment_name']
results_dir = f'results/notebooks/{exp_name}'
cache = storage_from_config(CONFIG['cache_backend_settings'])
shared_cache_info = CacheInformation(cache.capacity, cache.size)

caching_strategy, _, _ = strategies_from_config(CONFIG['cache_manager_settings'], results_dir)


19-04-14 18:34:04:INFO:Parsed state space definition: Floatbox((4,) <class 'numpy.float32'> )
19-04-14 18:34:04:INFO:Parsed action space definition: Intbox(() <class 'numpy.int32'> )
19-04-14 18:34:04:INFO:No preprocessing required.
19-04-14 18:34:04:INFO:Execution spec is: {'gpu_spec': {'gpus_enabled': False}, 'mode': 'single', 'distributed_spec': None, 'disable_monitoring': False, 'device_strategy': 'default', 'default_device': None, 'device_map': {}, 'session_config': {'type': 'monitored-training-session', 'allow_soft_placement': True, 'log_device_placement': False}, 'seed': None, 'enable_profiler': False, 'profiler_frequency': 1000, 'enable_timeline': False, 'timeline_frequency': 1}
19-04-14 18:34:04:WARNING:`device_map` given for device-strategy=`default`. Map will be ignored. Use device-strategy=`custom` together with a `device_map`.
19-04-14 18:34:04:INFO:Initializing graph executor with default device strategy. Backend will assign all visible devices.
19-04-14 18:34:04:INFO:GPU

/home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [26]:
key_should_be_cached = 'cache'
dummy_values =  {'field1': '2Ki"-2$>z\'087Vq.7d"+:)Ec1Iw7/`0Y5(#<%Fm"$:4%h6G\'\'0 !M=#"6=;j"C+86p;+>9Vy?L546z9@1=Ze3/`?^y0Vo7/|94n/',
             'field0': '(Ei#Lw*4404n0>z22<"]5)8*$/b;Q345~2Aq\':8!6:=Q7-Z1((l76j;%>)Du?A7,#f/1t*&d=Ke#:";@#,]o,A}.!b9Ic;(>17r;',
                 'field7': '=/>0Pu:M59P)$(>$G9);j"Ik(&,(44=;>1*,7Ai&Y+(#6)Ri/6"996)R#8!b?Ms\'#4-2.= |-(v+24\'Ge3E!35x\'+4![{#Tu(%"9', 
                 'field6': ',(d&\\!"?>5),3%(%Xw?D-$(n4Rw\'%./:,!Ls:7$(Ay9949@3(.f"_m7Sq/B=4X{;:.(?b"L%==8<[i802 62 ^q"?t6%09D5;E\' ', 
                 'field9': '(344S)-_%4:"\'_a6&. ?n*Ks:.p.>$6+t\' 8)A+-Ho)((9Ig,,4,?h-\'b/@m/>x&E74]{6A+1> 09~5[u7F%>4`+?64J#=485Ym2',
                 'field8': '&Ls.Z3,H90S!(Gc7?"#F)-Q97&v\'-"/ t*@\x7f>-z/Qs >&,,2=X1=M9&=v#9*#)|7^c [!)\\7&\'j3!n3$t?Ay\'U!4Hy%-4%[7>L)%',
                 'field3': "/.`;%h>&j/K3-G{7V%1Uy>'&$\\-4E%8(t+F= ? )-d%\\\x7f6R)9H)$A')I\x7f+<23Di/I7>Yc>W{$B5)/>3*z:Je 2`'5r+Uq>^?&Rs#", 
                 'field2': '7B; ,|,#|>:40Yc*^y6"*51p\'1z7[#-X=-Su$Pc"Y\x7f\'.d;Ma8\\-- ~7+,9+t$Y{/Mg"3l.@u1J!$?"(Gy2Fa:0d.Si92,4M32Cu#',
                 'field5': '+*6 \\g"9d/:: !:!$26 8 A+60l2 v 4n#P\'7^)!J72021\'t##r>;,8De;X--2b2*z$)* >f%@c3<85Ay37z9Aq/R5*O7) b3Ku>', 
                 'field4': '(Ca2<:*Hg7O/26l>-27;0<T)8Wm?.6+2*1^}>+ 0No;Ss2Dg>3h2]q!+*1Ec1Xg.]3&3~.?0\'<.$Ru3)j3,.?Q1-L!#Py\'&,"$v&'}


key_should_not_be_cached = 'dontcache'

In [29]:
for ep in range(10):
    for query in range(25):
        should = caching_strategy.should_cache(key_should_be_cached, dummy_values, 50, OperationType.New)
        if should:
            for i in range(5):
                caching_strategy.observe(key_should_be_cached, ObservationType.Hit, {})
            caching_strategy.observe(key_should_be_cached, ObservationType.Invalidate, {})
        else:
            caching_strategy.observe(key_should_be_cached, ObservationType.Miss, {})
    
    for query in range(25):
        caching_strategy.should_cache(key_should_not_be_cached, dummy_values, 50, OperationType.New)
        caching_strategy.observe(key_should_not_be_cached, ObservationType.Invalidate, {})
    
    shared_cache_info.hit = 30
    shared_cache_info.miss = 25
    caching_strategy.end_episode(shared_cache_info)

19-04-14 18:34:25:INFO:cache,False,True
19-04-14 18:34:25:INFO:1


InvalidArgumentError: indices[0] = 8191 is not in [0, 5000)
	 [[node prioritized-replay/GatherV2_1 (defined at /home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/rlgraph/components/component.py:1202) ]]

Caused by op 'prioritized-replay/GatherV2_1', defined at:
  File "/home/samialab/anaconda3/envs/rlcache/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/samialab/anaconda3/envs/rlcache/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/samialab/anac

In [20]:
# caching_strategy._incomplete_experiences.clear()
# caching_strategy.save_results()
# import numpy as np
# np.asarray(caching_strategy.losses)[1]

In [67]:
# Convert to unit tests?
from rlcache.strategies.caching_strategies.rl_caching_strategy import _MonitoringEntry
from time import sleep, time 
test = _MonitoringEntry('sad', False, False, time(), 0)
sleep(3)
test_2 = _MonitoringEntry('sad', False, False, time(), 0)

assert test != test_2